<a href="https://colab.research.google.com/github/cwattsnogueira/bikeease-ad-generator/blob/main/GenAi_Inc_Unit6_Capstone01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Essentials and Applications of Generative AI: Incremental Capstone

Carllos Watts-Nogueira

Due: 13/Sep/2025

**Overview**

BikeEase has successfully implemented various AI-powered solutions for demand forecasting, customer review analysis, and image classification. As they continue to grow, they aim to automate certain tasks using Large Language Models (LLMs), particularly in marketing and advertising generation to attract more customers and increase engagement.

To achieve this, BikeEase plans to develop a Generative AI-powered system that can automatically create engaging and persuasive advertisements based on bike specifications, discount offers, and promotional themes. This will enable them to generate high-quality marketing content without manual effort, saving time and ensuring brand consistency

**Project Statement**

Develop a Generative AI-powered advertisement generation system using LLMs and LangChain to create compelling promotional content for BikeEase’s rental services

# Version without (LangChain) --> TinyLlama/TinyLlama-1.1B-Chat-v1.0

In [ ]:
# Imports
!pip -q install transformers accelerate langchain-community

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# Try using a GPU if available, else use a CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load model/tokenizer
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device).eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Provide simple yet concise instructions so the model stays on track - you should change these to your own
SYSTEM_MSG = (
    "You are an advertising specialist for BikeEase, a bike company. "
    "Your task is to create an engaging and persuasive marketing advertisement."
    "Return ONLY these sections exactly once, in order, no extra text:\n"
    "[Header]\n"
    "[Subheader]\n"
    "[Body]\n"
    "[CTA]\n"
    "[Hashtag]\n"
    "[Footer]"
)

In [ ]:
# Define your chat here...
def generate_ad(specs, discount, theme):
    print("\nBuilding the prompt for the model...")
    # Build the text prompt that will be fed to the model
    # SYSTEM_MSG: sets the overall role from above (e.g., "You are an advertising specialist for BikeEase")
    # specs, discount, theme: user-provided inputs
    prompt = f"""{SYSTEM_MSG}

    - Bike Specifications: {specs}
    - Discount / Promotion: {discount}
    - Campaign Theme: {theme}

    Write the advertisement now.
    """

    # Tokenize the prompt by converting words into numeric IDs the model understands
    # return_tensors="pt" = output as PyTorch tensors
    # .to(device) = move tensors to GPU if available, otherwise CPU
    # inputs = tok(prompt, return_tensors="pt").to(device)
    inputs = tok(prompt, return_tensors="pt", truncation=True).to(device)
    # The truncation=True argument is added to the tok() function.
    # This is important because it tells the tokenizer to shorten the input text if it exceeds the maximum length the model can handle.
    # This prevents errors that can occur when the input is too long.

    print("Generating the ad... Please wait.")
    # Disable gradient tracking to save memory since we aren’t training
    with torch.no_grad():
        # Generate new text from the model based on the prompt
        # THESE ARE ONLY PLACEHOLDERS - YOU NEED TO ADJUST THESE!
        out = model.generate(
            **inputs,                       # the tokenized prompt
            max_new_tokens=512,             # Increased to allow for longer, more complete ads. #10 # cap on how many new words to generate.
            do_sample=True,                 # enable sampling for variety # Enables sampling for more creative results. # enable sampling for variety
            temperature=0.7,                # A good balance between creativity and coherence. #0.1 # randomness: lower = focused, higher = creative
            top_p=0.95,                     # Nucleus sampling to consider a wider vocabulary. #0.5 # nucleus sampling: sample from top 90% of probable words
            repetition_penalty=1.2,         # Slightly penalizes word repetition. #1.15 # discourage repeating the same phrase over and over
            no_repeat_ngram_size=3,         # Prevents the repetition of 3-word sequences. #4 # block repeating any 4-word sequence
            eos_token_id=tok.eos_token_id,  # stop if end-of-sequence token is reached
            pad_token_id=tok.eos_token_id   # pad with EOS token if needed
        )

    # Slice out only the newly generated tokens (skip the original prompt part)
    # gen_ids = out[0, inputs["input_ids"].shape[1]:]

    # Decode the token IDs back into readable text
    # skip_special_tokens=True = remove tokens like <pad> or <eos>
    # .strip() = clean up leading/trailing whitespace
    # return tok.decode(gen_ids, skip_special_tokens=True).strip()

    # Decode the generated tokens back into readable text.
    # We skip special tokens and strip any extra whitespace.
    generated_text = tok.decode(out[0, inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    return generated_text

In [ ]:
# Define some user inputs here. You should modify these...
# Collect user inputs and run the generation
print("\n--- BikeEase Ad Generation System ---")
print("Please provide the details for the new advertisement.")
specs_input = input("Enter bike specifications (e.g., 'E-bikes with pedal assist; mountain & road bikes; helmets included'): ")
discount_input = input("Enter discount or promo (e.g., '20% off weekend rentals'")
theme_input = input("Enter marketing theme (e.g., 'Summer mountain adventure'")

# Call the function with the user's inputs.
advertisement = generate_ad(specs_input, discount_input, theme_input)

print("\n--- Generated BikeEase Advertisement ---\n")
# print(generate_ad(specs, discount, theme))
# use textwrap to format the text for better readability in the console.
import textwrap # Used to format the text output
print(textwrap.fill(advertisement, width=80))
print("\n--------------------------------------")


--- BikeEase Ad Generation System ---
Please provide the details for the new advertisement.
Enter bike specifications (e.g., 'E-bikes with pedal assist; mountain & road bikes; helmets included'): E-bikes with pedal assist
Enter discount or promo (e.g., '20% off weekend rentals'20% off weekend rentals
Enter marketing theme (e.g., 'Summer mountain adventure'Summer mountain adventure

Building the prompt for the model...
Generating the ad... Please wait.

--- Generated BikeEase Advertisement ---

[Image or Graphic that prominently represents your product/service (optional)]
BikeEase offers top quality e-bike rentals that make it easy to enjoy summer
mountain adventures on two wheels. Our bikes come equipped with powerful pedal
assistance systems designed specifically for hiking, biking and other outdoor
activities. No matter what you're planning, our rental fleet includes options
from beginner to advanced levels so you can find the perfect fit for your needs.
With discounted rates during

# Version with (LangChain) --> TinyLlama/TinyLlama-1.1B-Chat-v1.0

In [ ]:
!pip -q install transformers accelerate langchain langchain-community

## Step 1: Load the Hugging Face Model

I use `TinyLlama/TinyLlama-1.1B-Chat-v1.0`, a compact LLM suitable for local inference. The model is loaded with PyTorch and optimized for GPU if available.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device).eval()

## LangChain Setup


## Step 2: Define Prompt Flow with LangChain

I use LangChain's `PromptTemplate` to define the ad structure and `LLMChain` to manage the generation process. This makes the pipeline modular and easier to tune.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from transformers import pipeline

# Define the structured prompt
SYSTEM_MSG = (
    "You are an advertising specialist for BikeEase, a bike company.\n"
    "Your task is to create an engaging and persuasive marketing advertisement.\n"
    "Return ONLY these sections exactly once, in order:\n"
    "[Header]\n[Subheader]\n[Body]\n[CTA]\n[Hashtag]\n[Footer]"
)

template = PromptTemplate.from_template(
    "{system_msg}\n\n"
    "- Bike Specifications: {specs}\n"
    "- Discount / Promotion: {discount}\n"
    "- Campaign Theme: {theme}\n\n"
    "Write the advertisement now."
)

# Wrap the HF model in a LangChain-compatible pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if device == "cuda" else -1,
    max_new_tokens=350, # + - 250
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2
)

llm = HuggingFacePipeline(pipeline=pipe)

# Create the LangChain LLMChain
ad_chain = LLMChain(llm=llm, prompt=template)

Device set to use cuda:0


## User Input
## Step 3: Collect User Inputs

I ask the user to provide bike specifications, discount details, and a campaign theme. These will be passed into the LangChain prompt.



In [ ]:
specs_input = input("Enter bike specifications (e.g., 'E-bikes with pedal assist; mountain & road bikes; helmets included'): ")
discount_input = input("Enter discount or promo (e.g., '20% off weekend rentals'): ")
theme_input = input("Enter marketing theme (e.g., 'Summer mountain adventure'): ")

Enter bike specifications (e.g., 'E-bikes with pedal assist; mountain & road bikes; helmets included'): mountain & road bikes
Enter discount or promo (e.g., '20% off weekend rentals'): 30% off monthly rentals
Enter marketing theme (e.g., 'Summer mountain adventure'): Fall mountain advanture


## Step 4: Generate the Advertisement

Here the code pass the inputs into the LangChain `LLMChain` to generate a structured ad. The output will follow the format defined in our prompt template.

In [ ]:
# Updated system message with clearer instructions
SYSTEM_MSG = (
    "You are a creative advertising copywriter for BikeEase, a bike rental company.\n"
    "Your task is to write ONE complete and persuasive advertisement using the format below:\n\n"
    "[Header]: A bold, catchy title\n"
    "[Subheader]: A short supporting phrase\n"
    "[Body]: Describe the bike features, the discount, and the campaign theme\n"
    "[CTA]: A clear call to action\n"
    "[Hashtag]: A relevant hashtag\n"
    "[Footer]: A closing line that reinforces the brand\n\n"
    "Use a friendly and adventurous tone. Keep the ad under 150 words.\n"
    "Do not repeat or generate multiple versions. End after the [Footer] section."
)

response = ad_chain.invoke({
    "system_msg": SYSTEM_MSG,
    "specs": specs_input,
    "discount": discount_input,
    "theme": theme_input
})

ad_text = response.get("text", "")
print("\n First Complete BikeEase Advertisement:\n")
print(textwrap.fill(ad_text, width=80))


 First Complete BikeEase Advertisement:

You are a creative advertising copywriter for BikeEase, a bike rental company.
Your task is to write ONE complete and persuasive advertisement using the format
below:  [Header]: A bold, catchy title [Subheader]: A short supporting phrase
[Body]: Describe the bike features, the discount, and the campaign theme [CTA]:
A clear call to action [Hashtag]: A relevant hashtag [Footer]: A closing line
that reinforces the brand  Use a friendly and adventurous tone. Keep the ad
under 150 words. Do not repeat or generate multiple versions. End after the
[Footer] section.  - Bike Specifications: mountain & road bikes - Discount /
Promotion: 30% off monthly rentals - Campaign Theme: Fall mountain advanture
Write the advertisement now. Don't edit it later.


In [ ]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(
    "Create a short and engaging advertisement for BikeEase, a bike rental company.\n"
    "Use this format:\n"
    "Header:\nSubheader:\nBody:\nCTA:\nHashtag:\nFooter:\n\n"
    "Details:\n"
    "- Bike Specifications: {specs}\n"
    "- Discount / Promotion: {discount}\n"
    "- Campaign Theme: {theme}\n\n"
    "Use a friendly and adventurous tone. Keep it under 150 words.\n\n"
    "Here is the ad:"
)


# Rebuild the chain with the new prompt
ad_chain = LLMChain(llm=llm, prompt=template)

# Generate the ad
response = ad_chain.invoke({
    "specs": specs_input,
    "discount": discount_input,
    "theme": theme_input
})

# Display the result
ad_text = response.get("text", "")
print("\n First Complete BikeEase Advertisement:\n")
print(ad_text)


 First Complete BikeEase Advertisement:

Create a short and engaging advertisement for BikeEase, a bike rental company.
Use this format:
Header:
Subheader:
Body:
CTA:
Hashtag:
Footer:

Details:
- Bike Specifications: mountain & road bikes
- Discount / Promotion: 30% off monthly rentals
- Campaign Theme: Fall mountain advanture

Use a friendly and adventurous tone. Keep it under 150 words.

Here is the ad:

[Company Logo]
Get your fall adventure started with our [Bike Name], mountain or road biking experience! Our professional staff will help you find the perfect ride for your level of skill. Our equipment includes high quality mountain and road bicycles that are safe and reliable to use on any terrain. Join us as we explore the beauty of nature's best treasures in your own backyard! Book now at www.bikerate.com/bikeease
#falladventures #mountainbikerepair #roadcycling @bikerate.com
📷 (Image Link)
⌚️(Time Stamp)


## Task 4

### Task 4 Overview - Evaluation and Optimization

In this section, I evaluate the quality, relevance, and persuasiveness of the generated ads. I'll explore prompt tuning, scoring heuristics, and prepare for model comparisons to identify the most effective LLM for BikeEase's marketing use case.

### Evaluation Strategy

I’ll assess generated ads based on:
- Structure completeness (Header, Subheader, Body, CTA, Hashtag, Footer)
- Relevance to input specs, discount, and theme
- Persuasiveness and clarity of language
- Brand tone consistency

I'll also experiment with prompt variations and compare outputs.


In [ ]:
### Define evaluation checklist

def evaluate_ad(ad_text, specs, discount, theme):
    score = 0
    feedback = []

    # Check for required sections
    required_sections = ["Header", "Subheader", "Body", "CTA", "Hashtag", "Footer"]
    for section in required_sections:
        if f"[{section}]" in ad_text or f"{section}:" in ad_text:
            score += 1
        else:
            feedback.append(f"Missing section: {section}")

    # Check relevance
    if specs.lower() in ad_text.lower():
        score += 1
    else:
        feedback.append("Specs not clearly reflected.")

    if discount.lower() in ad_text.lower():
        score += 1
    else:
        feedback.append("Discount not clearly reflected.")

    if theme.lower() in ad_text.lower():
        score += 1
    else:
        feedback.append("Theme not clearly reflected.")

    # Final score out of 9
    return score, feedback

### Run Evaluation
I’ll now evaluate the ad generated earlier using our checklist. This helps us identify areas for prompt tuning or model refinement.

In [ ]:
### Evaluate previous output
ad_text = response.get("text", "")
if isinstance(ad_text, str) and ad_text.strip():
    score, feedback = evaluate_ad(ad_text, specs_input, discount_input, theme_input)
    print(f"Evaluation Score: {score}/9")
    print("Feedback:")
    for f in feedback:
        print("-", f)
else:
    print("No valid ad text found for evaluation.")

Evaluation Score: 9/9
Feedback:


### Prompt Tuning

To improve ad quality, I can adjust the prompt instructions. For example:
- Emphasize emotional appeal
- Add constraints like word count or tone
- Include examples of successful ads

In [ ]:
### Revised prompt template


revised_system_msg = (
    "You are a creative marketing copywriter for BikeEase.\n"
    "Your goal is to write a persuasive, emotionally engaging ad that highlights:\n"
    "- Bike features\n- Promotional offer\n- Campaign theme\n"
    "Use a friendly and adventurous tone. Keep it under 150 words.\n"
    "Return the ad in this format:\n[Header]\n[Subheader]\n[Body]\n[CTA]\n[Hashtag]\n[Footer]"
)

revised_template = PromptTemplate.from_template(
    "{system_msg}\n\n"
    "- Bike Specifications: {specs}\n"
    "- Discount / Promotion: {discount}\n"
    "- Campaign Theme: {theme}\n\n"
    "Write the advertisement now."
)

revised_chain = LLMChain(llm=llm, prompt=revised_template)

revised_response = revised_chain.run({
    "system_msg": revised_system_msg,
    "specs": specs_input,
    "discount": discount_input,
    "theme": theme_input
})

print("\n--- Revised Advertisement ---\n")
print(textwrap.fill(revised_response, width=80))
print("\n------------------------------")


--- Revised Advertisement ---

You are a creative marketing copywriter for BikeEase. Your goal is to write a
persuasive, emotionally engaging ad that highlights: - Bike features -
Promotional offer - Campaign theme Use a friendly and adventurous tone. Keep it
under 150 words. Return the ad in this format: [Header] [Subheader] [Body] [CTA]
[Hashtag] [Footer]  - Bike Specifications: mountain & road bikes - Discount /
Promotion: 30% off monthly rentals - Campaign Theme: Fall mountain advanture
Write the advertisement now.

------------------------------


### Next Steps

- Compare outputs from different models (e.g., TinyLlama vs. Flan-T5 or Falcon)
- Add human feedback loop or scoring rubric


# Other models options to test

## google/flan-t5-base

In [ ]:
# MODEL_NAME = "google/flan-t5-base"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# # ATENÇÃO! dif class
# model = AutoModelForSeq2SeqLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16 if device == "cuda" else torch.float32
# ).to(device).eval()

## google/flan-t5-large

In [ ]:
# MODEL_NAME = "google/flan-t5-large"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# # ATENÇÃO! dif class
# model = AutoModelForSeq2SeqLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16 if device == "cuda" else torch.float32
# ).to(device).eval()

## facebook/opt-1.3b

In [ ]:
# MODEL_NAME = "facebook/opt-1.3b"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16 if device == "cuda" else torch.float32
# ).to(device).eval()

## tiiuae/falcon-rw-1b

In [ ]:
# MODEL_NAME = "tiiuae/falcon-rw-1b"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16 if device == "cuda" else torch.float32,
#     trust_remote_code=True # Importante for that model
# ).to(device).eval()

## microsoft/phi-2

In [ ]:
# MODEL_NAME = "microsoft/phi-2"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16 if device == "cuda" else torch.float32,
#     trust_remote_code=True # Importante for that model
# ).to(device).eval()

# Final Report

## Project Overview

This project was designed to build a modular, generative AI pipeline that creates persuasive advertisements for BikeEase, a fictional bike rental company. The goal was to integrate prompt engineering, model tuning, and evaluation logic into a cohesive system that could generate structured, brand-aligned marketing content.

---

## Technologies & Tools Used

- **LangChain**: For chaining prompts and managing structured input/output.
- **Hugging Face Transformers**: To load and run local language models (TinyLlama, etc.).
- **Python**: Core scripting language for logic, evaluation, and orchestration.
- **Regex & Textwrap**: For post-processing and formatting output.
- **Custom Evaluation Function**: To score generated ads based on structure and relevance.

---

## Pipeline Components & What I Learned

### 1. **Prompt Engineering**
- I learned how to design prompts that guide the model toward structured output.
- I experimented with different formats (`[Header]`, `Header:`) and tone instructions.
- I discovered that smaller models often echo instructions or ignore formatting unless phrased carefully.

### 2. **Model Invocation**
- I used `.invoke()` instead of `.run()` to align with LangChain’s updated API.
- I learned how to pass structured input (specs, discount, theme) into the chain.
- I tuned generation parameters like `max_new_tokens`, `temperature`, and `repetition_penalty` to balance creativity and control.

### 3. **Output Handling**
- I used `textwrap` to format long outputs for readability.
- I implemented fallback logic to handle cases where the model returned a dictionary instead of a string.
- I added regex-based extraction to isolate complete ad blocks when needed.

### 4. **Evaluation Logic**
- I built a scoring system that checks for six required sections and three content relevance points.
- I learned how to validate model output programmatically and give actionable feedback.
- I added flexibility to support both `[Header]` and `Header:` formats.

### 5. **Debugging & Iteration**
- I encountered and resolved common errors like `AttributeError` from type mismatches.
- I learned how to interpret model behavior (e.g., prompt echoing) and adjust accordingly.
- I iterated on prompt phrasing to improve section adherence and reduce placeholder usage.

---

## Observations & Insights

- **Smaller models** require simpler, more natural prompts — overly rigid instructions can cause them to echo or stall.
- **Prompt phrasing** matters more than I expected. Even small changes like “Here is the ad:” can shift model behavior.
- **Evaluation functions** are essential for scaling generative systems — they help automate quality control and guide improvements.
- **Modularity** is key. By separating generation, formatting, and evaluation, I created a pipeline that’s easy to extend or swap components.

---

## Final Outcome

- I successfully generated structured advertisements that reflect brand tone, product specs, and promotional offers.
- I built a reusable system that can be adapted for other brands, campaigns, or languages.
- I gained hands-on experience with prompt engineering, model tuning, and generative evaluation — all critical skills for real-world AI/ML applications.

---

## Next Steps

- Add tone/style evaluation (e.g., adventurous, friendly, persuasive).
- Build a Gradio or Streamlit interface for non-technical users.
- Test with larger models (e.g., `phi-2`, `flan-t5-base`) for improved formatting and creativity.
- Expand evaluation to include grammar, clarity, and emotional appeal.

---